In [53]:
from datetime import datetime
from typing import Optional, Tuple, Any, List

class Reserve:
    def __init__(self, s_list: dict):
        '''
        학번과 군번이 서로 매핑된 데이터를 가지고 있다고 가정 (이를 통해 학생정보와 군정보를 연계)  
        '''
        self.data = []
        self.s_list = s_list  # {학번 : 군번}

    def is_available(self, x, y, z) -> int:
        '''
        군복무 가능여부를 판단하기 위한 함수  
        '''
        return x*y*z
    
    def binary_search(self, target, start, end, find_insert_pos=False) -> Optional[Tuple[int, Any]]:
        '''
        이진 탐색 메소드
        '''
        if end < start:
            if find_insert_pos:
                return (start, None)  # 삽입 위치 반환
            return None
        mid = (start + end)//2
        mid_data = self.data[mid]
        if mid_data[0] == target:       # 찾으려는 학번과 일치한다면
            if find_insert_pos:
                return None
            return (mid, mid_data)      # 해당 데이터와 인덱스 번호 추출
        elif mid_data[0] > target:
            return self.binary_search(target, start, mid-1, find_insert_pos)
        elif mid_data[0] < target:
            return self.binary_search(target, mid+1, end, find_insert_pos)

    def init_create(self, student_data: dict, military_data: dict) -> None:
        '''
        학생정보와 군정보를 받아 첫 예비군 데이터를 생성하는 함수\n 
        학생정보와 군정보는 여러 학생 정보가 담긴 딕셔너리 형태  
        '''
        for s_num, s_data in student_data.items():

            temp_data = []                          # 정보저장을 위한 임시 리스트

            if s_data[2] == 1:                      # 학생이 여자이면 추가 X
                continue

            if s_num in self.s_list.keys():         # 학번을 통해 군번 연결 (학번에 해당하는 군번이 존재하지 않을 경우 건너뛰기)
                m_num = self.s_list[s_num]
                temp_data.append(s_num)             # 임시 데이터에 학번 추가
                temp_data.append(m_num)             # 임시 데이터에 군번 추가
                temp_data.append(s_data[0])             # 임시 데이터에 이름 추가
            else:
                continue

            is_available = self.is_available(s_data[3], s_data[4], s_data[5])   # 예비군 훈련 가능여부 판단
            temp_data.append(is_available)          # 예비군 훈련 가능여부 추가

            m_data = military_data[m_num]           # 연결된 군번에 해당하는 데이터 추출
            out_date = m_data[4]                    # 전역일 데이터 추출
            out_year = int(out_date.split('-')[0])  # 전역년도 추출
            current_year = datetime.today().year    # 현재년도 추출
            reserve_year = current_year - out_year  # 예비군 복무연차 계산 (현재년도 - 전역년도)
            temp_data.append(reserve_year)          # 예비군 복무연차 추가

            temp_data.append(0)                     # 예비군 훈련 횟수(처음 생성 시 0으로 초기화)
            temp_data.append(0)                     # 예비군 훈련 시간(처음 생성 시 0으로 초기화)
            temp_data.append(6)                     # 훈련 연기 가능 횟수(처음 생성 시 6으로 초기화)
            temp_data.append(0)                     # 조기 퇴소 횟수(처음 생성 시 0으로 초기화)

            self.data.append(temp_data)             # 임시 데이터를 최종 데이터에 추가
            self.data.sort(key = lambda x : x[0])   # 효율적인 검색/업데이트를 위한 정렬

    def create(self, student_data: dict, military_data: dict) -> Optional[List]:
        '''
        학생정보와 군정보를 받아 새로운 예비군 데이터를 생성하는 함수\n 
        학생정보와 군정보는 하나의 학생 정보가 담긴 딕셔너리 형태(개별 생성)  
        '''
        s_num, s_data = list(student_data.items())[0]
        m_num, m_data = list(military_data.items())[0]

        temp_data = []                          # 정보저장을 위한 임시 리스트

        if s_data[2] == 1:                      # 학생이 여자이면 추가 X
            return None

        temp_data.append(s_num)                 # 임시 데이터에 학번 추가
        temp_data.append(m_num)                 # 임시 데이터에 군번 추가
        temp_data.append(s_data[0])             # 임시 데이터에 이름 추가

        is_available = self.is_available(s_data[3], s_data[4], s_data[5])   # 예비군 훈련 가능여부 판단
        temp_data.append(is_available)          # 예비군 훈련 가능여부 추가

        out_date = m_data[4]                    # 전역일 데이터 추출
        out_year = int(out_date.split('-')[0])  # 전역년도 추출
        current_year = datetime.today().year    # 현재년도 추출
        reserve_year = current_year - out_year  # 예비군 복무연차 계산 (현재년도 - 전역년도)
        temp_data.append(reserve_year)          # 예비군 복무연차 추가

        temp_data.append(0)                     # 예비군 훈련 횟수(처음 생성 시 0으로 초기화)
        temp_data.append(0)                     # 예비군 훈련 시간(처음 생성 시 0으로 초기화)
        temp_data.append(6)                     # 훈련 연기 가능 횟수(처음 생성 시 6으로 초기화)
        temp_data.append(0)                     # 조기 퇴소 횟수(처음 생성 시 0으로 초기화)

        result = self.binary_search(s_num, 0, len(self.data)-1, find_insert_pos=True)
        if result:
            insert_pos, _ = result
        else:
            insert_pos = len(self.data)
            
        self.data.insert(insert_pos, temp_data) # 임시 데이터를 최종 데이터에 추가(정렬 형식에 맞추어서 추가)
       
        return [temp_data]

    def retrieve(self, value = None, is_filter = False) -> Optional[List]:
        '''
        데이터를 검색하기 위한 함수\n
        is_filter가 True이면 이번학기에 예비군 훈련이 가능한 학생 필터링\n
        is_filter가 False이면 학번으로 특정 학생의 예비군 정보 검색  
        '''
        found_data = []
        if is_filter:                               # 필터링 적용
            for data in self.data:
                if data[3] == 1:
                    found_data.append(data)
        else:                                       # 특정 학생 검색
            s_num = value
            result = self.binary_search(s_num, 0, len(self.data)-1)
            if result:
                _, data = result
            found_data.append(data)
        return found_data
    
    def update_after_train(self, update_list: list) -> Optional[List]:
        '''
        예비군 수료 정보 업데이트 함수\n
        업데이트 대상이 되는 예비군 수료 정보는\n
        [학번, 예비군 훈련 여부, 훈련 시간, 훈련 연기 여부, 조기퇴소 여부] 라고 가정\n
        이진 탐색으로 학번에 해당하는 데이터를 찾은 뒤, 정보 업데이트
        '''
        return_data = []
        for update_data in update_list:
            result = self.binary_search(update_data[0], 0, len(self.data)-1)
            if result:
                idx, data = result
            else:
                continue
            data[5] += update_data[1]               # 예비군 훈련 여부 업데이트
            data[6] += update_data[2]               # 예비군 훈련 시간 업데이트
            data[7] -= update_data[3]               # 예비군 훈련 연기 여부 업데이트
            data[8] += update_data[4]               # 예비군 조기퇴소 여부 업데이트
            self.data[idx] = data
            return_data.append(data)
        return return_data

    def update_is_available(self, student_data : dict) -> Optional[List]:
        '''
        학생정보가 수정되었을 때, 업데이트 함수\n
        교환/유학 여부 수정 시 업데이트\n
        학생정보는 동일하게 dict 형식으로 받음
        '''
        return_data = []
        for s_num, s_data in student_data.items():
            is_available = self.is_available(s_data[3], s_data[4], s_data[5])
            result = self.binary_search(s_num, 0, len(self.data)-1)
            if result:
                idx, data = result
            data[3] = is_available
            self.data[idx] = data
            return_data.append(data)
        return return_data
    
    def auto_delete(self, student_data: dict) -> list:
        """
        조건에 따라 예비군 정보를 자동 삭제하는 함수

        삭제 조건:
        - 학생정보에 학번이 없음 (졸업/수료/자퇴)
        - 학기가 9 이상 (추가학기)
        - 예비군 연차가 5년 이상
        """
        deleted = []

        # 복사본으로 순회하면서 원본 리스트에서 삭제
        for entry in self.data[:]:
            student_id = entry[0]
            reserve_year = entry[4]

            # 조건 1: 학번이 student_data에 없음
            if student_id not in student_data:
                self.data.remove(entry)
                deleted.append(entry)
                continue

            # 조건 2: 학기가 9 이상
            semester = student_data[student_id][6]  
            if semester >= 9:
                self.data.remove(entry)
                deleted.append(entry)
                continue

            # 조건 3: 예비군 연차가 5 이상
            if reserve_year >= 5:
                self.data.remove(entry)
                deleted.append(entry)
                continue

        print(f"[자동삭제 완료] 삭제된 예비군 데이터 {len(deleted)}건")
        return deleted

    def manual_delete(self, student_ids: list) -> list:
        """
        사용자가 직접 지정한 학번 목록에 대해 예비군 데이터를 수동 삭제하는 함수
        """
        deleted = []

        for s_num in student_ids:
            result = self.binary_search(s_num, 0, len(self.data) - 1)
            if result:
                idx, data = result
                del self.data[idx]
                deleted.append(data)
                print(f"[-] 학번 {s_num} 예비군 정보 삭제 완료.")
            else:
                print(f"[!] 학번 {s_num} 예비군 정보를 찾을 수 없습니다.")

        return deleted



In [54]:
import pickle
with open('../data/student_data.pickle', 'rb') as fr:
    student_data = pickle.load(fr)
with open('../data/military_data.pickle', 'rb') as fr:
    military_data = pickle.load(fr)
with open('../data/s_list.pickle', 'rb') as fr:
    s_list = pickle.load(fr)

In [55]:
student_data

{'2021549578': ['홍길동', 20, 0, 0, 1, 1, 5],
 '2019042697': ['김현우', 26, 0, 1, 1, 1, 4],
 '2022460122': ['강민준', 24, 0, 1, 1, 0, 7],
 '2021188984': ['장도윤', 23, 0, 1, 1, 1, 6],
 '2018448337': ['김현우', 23, 0, 1, 1, 1, 7],
 '2020454347': ['정민준', 25, 0, 1, 1, 1, 2],
 '2022327139': ['이지훈', 23, 0, 1, 1, 1, 7],
 '2021291417': ['김현우', 24, 0, 1, 1, 1, 2],
 '2020369549': ['윤도윤', 23, 0, 1, 1, 1, 2],
 '2021410468': ['정도윤', 23, 0, 1, 1, 1, 3],
 '2021797494': ['윤민준', 25, 0, 1, 1, 1, 5],
 '2019747283': ['강도윤', 23, 0, 1, 1, 1, 2],
 '2021712537': ['임지후', 24, 0, 1, 1, 1, 2],
 '2020681376': ['강하준', 24, 0, 1, 1, 1, 5],
 '2021914660': ['정현우', 25, 0, 1, 1, 1, 7],
 '2021906093': ['정주원', 24, 0, 0, 1, 1, 6],
 '2018599495': ['조건우', 24, 0, 1, 1, 1, 4],
 '2020225223': ['윤지훈', 23, 0, 1, 1, 1, 5],
 '2019364234': ['정현우', 25, 0, 1, 0, 1, 5],
 '2018440170': ['강하준', 23, 0, 1, 1, 0, 5],
 '2018695099': ['조현우', 23, 0, 1, 1, 1, 7],
 '2021588001': ['강민준', 24, 0, 1, 1, 1, 7],
 '2021985653': ['임현우', 24, 0, 1, 1, 1, 6],
 '201979736

In [56]:
new_student_data = {'2021657987':['경민서', 24, 0, 1, 1, 1, 8]}
new_military_data = {'21-71357927':['육군','병장','25사단','영상감시병','2022-09-15']}
update_list = [['2021657987', 1, 8, 0, 0]]

reserve = Reserve(s_list)
reserve.init_create(student_data, military_data)
a = reserve.create(new_student_data, new_military_data)
a

[['2021657987', '21-71357927', '경민서', 1, 3, 0, 0, 6, 0]]

In [57]:
reserve.retrieve('2021657987')

[['2021657987', '21-71357927', '경민서', 1, 3, 0, 0, 6, 0]]

In [58]:
reserve.update_after_train(update_list)
reserve.retrieve('2021657987')

[['2021657987', '21-71357927', '경민서', 1, 3, 1, 8, 6, 0]]

In [59]:
renew_stduent_data = {'2021657987':['경민서', 24, 0, 1, 0, 1, 8]}
reserve.update_is_available(renew_stduent_data)
reserve.retrieve('2021657987')

[['2021657987', '21-71357927', '경민서', 0, 3, 1, 8, 6, 0]]

In [48]:
deleted_data = reserve.auto_delete(student_data)
print(deleted_data)

[자동삭제 완료] 삭제된 예비군 데이터 17건
[['2018039079', '19-75675132', '임하준', 1, 5, 0, 0, 6, 0], ['2018265293', '19-87182137', '강현우', 0, 5, 0, 0, 6, 0], ['2018359131', '19-87976763', '강현우', 1, 5, 0, 0, 6, 0], ['2018438911', '19-08451645', '임지후', 0, 5, 0, 0, 6, 0], ['2018448337', '19-84155259', '김현우', 1, 5, 0, 0, 6, 0], ['2018451975', '19-16721308', '최건우', 1, 5, 0, 0, 6, 0], ['2018500305', '19-15505461', '장지훈', 1, 5, 0, 0, 6, 0], ['2018574745', '19-78423629', '이지후', 1, 5, 0, 0, 6, 0], ['2018595286', '19-91806045', '강건우', 1, 5, 0, 0, 6, 0], ['2018599495', '19-54339766', '조건우', 1, 5, 0, 0, 6, 0], ['2018695099', '19-45529064', '조현우', 1, 5, 0, 0, 6, 0], ['2018710124', '19-72734014', '김민준', 0, 5, 0, 0, 6, 0], ['2018776863', '19-26208659', '강건우', 0, 5, 0, 0, 6, 0], ['2018890036', '19-42315773', '장하준', 1, 5, 0, 0, 6, 0], ['2018923873', '19-46933146', '최민준', 1, 5, 0, 0, 6, 0], ['2018989905', '19-34213720', '이현우', 1, 5, 0, 0, 6, 0], ['2021657987', '21-71357927', '경민서', 0, 3, 1, 8, 6, 0]]


In [49]:
reserve.retrieve('2021657987')

UnboundLocalError: cannot access local variable 'data' where it is not associated with a value

In [60]:
student_ids = input("삭제할 학번을 입력하세요 (공백으로 구분): ").split()
reserve.manual_delete(student_ids)

[-] 학번 2021657987 예비군 정보 삭제 완료.


[['2021657987', '21-71357927', '경민서', 0, 3, 1, 8, 6, 0]]

In [61]:
reserve.retrieve('2021657987')

UnboundLocalError: cannot access local variable 'data' where it is not associated with a value